In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
import csv
from PIL import Image
import shutil
from tqdm import tqdm
import torch

from utils.utils0 import tensor_affine_transform, transform_to_displacement_field
from utils.utils1 import transform_points_DVF, ModelParams
from utils.SuperPoint import SuperPointFrontend, PointTracker
from utils.datagen import datagen

nn_thresh = 0.7
superpoint = SuperPointFrontend('utils/superpoint_v1.pth', nms_dist=4,
                          conf_thresh=0.015, nn_thresh=nn_thresh, cuda=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Device: cuda


In [2]:
model_params = ModelParams(sup=0, dataset=0, image=1, heatmaps=0, 
                           loss_image=1, num_epochs=10, learning_rate=1e-4)
model_params.print_explanation()

Model name:  dataset0_sup0_image1_heatmaps0_loss_image1
Model code:  00101_0.0001_0_10_1
Model params:  {'dataset': 0, 'sup': 0, 'image': 1, 'heatmaps': 0, 'loss_image_case': 1, 'loss_image': NCC(), 'loss_affine': None, 'learning_rate': 0.0001, 'decay_rate': 0.96, 'start_epoch': 0, 'num_epochs': 10, 'batch_size': 1, 'model_name': 'dataset0_sup0_image1_heatmaps0_loss_image1'}

Model name:  dataset0_sup0_image1_heatmaps0_loss_image1
Model code:  00101_0.0001_0_10_1
Dataset used:  Actual eye
Supervised or unsupervised model:  Unsupervised
Image type:  Image used
Heatmaps used:  Heatmaps not used
Loss function case:  1
Loss function for image:  NCC()
Loss function for affine:  None
Learning rate:  0.0001
Decay rate:  0.96
Start epoch:  0
Number of epochs:  10
Batch size:  1




In [3]:
train_dataset, train_df, train_path = datagen(model_params.dataset, True, model_params.sup)
test_dataset, test_df, test_path = datagen(model_params.dataset, False, model_params.sup)

Training eye dataset
Number of training data:  500
Test eye dataset
Number of testing data:  100


In [5]:
train_df.head()

,Unnamed: 0,source,Source ROI,target,Target ROI,training,Warped target images,Warped target ROI,Execution time,Directory
0,1337,Dataset/Dataset-processed/10-08-2562/3092607-C...,NaN,Dataset/Dataset-processed/10-08-2562/3092607-C...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/10-08-2562/3092607-C...
1,2957,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...
2,859,Dataset/Dataset-processed/23-12-2559/1426104/b...,NaN,Dataset/Dataset-processed/23-12-2559/1426104/b...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/23-12-2559/1426104/b1
3,3566,Dataset/Dataset-processed/13-01-2560/2446177/c...,NaN,Dataset/Dataset-processed/13-01-2560/2446177/c...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/13-01-2560/2446177/c
4,582,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...


In [6]:
test_df.head()

,Unnamed: 0,source,Source ROI,target,Target ROI,training,Warped target images,Warped target ROI,Execution time,Directory
500,3089,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...
501,760,Dataset/Dataset-processed/30-12-2559/3236468/b...,NaN,Dataset/Dataset-processed/30-12-2559/3236468/b...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/30-12-2559/3236468/b
502,3423,Dataset/Dataset-processed/20-01-2560/3755654/b...,NaN,Dataset/Dataset-processed/20-01-2560/3755654/b...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/20-01-2560/3755654/b
503,3062,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...
504,2108,Dataset/Dataset-processed/10-08-2562/2960420-C...,NaN,Dataset/Dataset-processed/10-08-2562/2960420-C...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/10-08-2562/2960420-C...


In [11]:
keypoints_path = []

In [12]:
train_bar = tqdm(train_dataset, total=len(train_dataset), desc='Train')
for i, data in enumerate(train_bar):

    # Get images and affine parameters
    if model_params.sup:
        source_image, target_image, affine_params_true = data
        affine_params_true = torch.tensor(affine_params_true)
    else:
        source_image, target_image = data
        affine_params_true = None
    source_image = source_image.to(device)
    target_image = target_image.to(device)

    points1, desc1, heatmap1 = superpoint(source_image[0, 0, :, :].cpu().numpy())
    points2, desc2, heatmap2 = superpoint(target_image[0, 0, :, :].cpu().numpy())

    tracker = PointTracker(5, nn_thresh=0.7)
    try:
        matches = tracker.nn_match_two_way(desc1, desc2, nn_thresh=nn_thresh)
    except:
        print('No matches found')
        # TODO: find a better way to do this
        pass

    matches1 = np.array(points1[:2, matches[0, :].astype(int)])
    matches2 = np.array(points2[:2, matches[1, :].astype(int)])
    
    # fig, ax = plt.subplots(1, 2)
    # ax[0].imshow(source_image[0, 0, :, :].cpu().detach().numpy())
    # ax[0].plot(matches1[0, :], matches1[1, :], 'r.')
    # ax[1].imshow(target_image[0, 0, :, :].cpu().detach().numpy())
    # ax[1].plot(matches2[0, :], matches2[1, :], 'g.')
    # ax[1].plot(matches1_2[0, :], matches1_2[1, :], 'r.')
    # plt.show()

    # create a dataframe with the matches
    df = pd.DataFrame({'x1': matches1[0, :], 'y1': matches1[1, :],
                       'x2': matches2[0, :], 'y2': matches2[1, :]})
    save_name = 'Dataset/Dataset-processed/' + f'img_{i}.csv'
    # print(save_name)
    keypoints_path.append(save_name)
    df.to_csv(save_name, index=False)

Train: 100%|██████████| 500/500 [01:18<00:00,  6.38it/s]


In [13]:
test_bar = tqdm(test_dataset, total=len(test_dataset), desc='Test')
for i, data in enumerate(test_bar):

    # Get images and affine parameters
    if model_params.sup:
        source_image, target_image, affine_params_true = data
        affine_params_true = torch.tensor(affine_params_true)
    else:
        source_image, target_image = data
        affine_params_true = None
    source_image = source_image.to(device)
    target_image = target_image.to(device)

    points1, desc1, heatmap1 = superpoint(source_image[0, 0, :, :].cpu().numpy())
    points2, desc2, heatmap2 = superpoint(target_image[0, 0, :, :].cpu().numpy())

    tracker = PointTracker(5, nn_thresh=0.7)
    try:
        matches = tracker.nn_match_two_way(desc1, desc2, nn_thresh=nn_thresh)
    except:
        print('No matches found')
        # TODO: find a better way to do this
        pass

    matches1 = np.array(points1[:2, matches[0, :].astype(int)])
    matches2 = np.array(points2[:2, matches[1, :].astype(int)])
    
    # fig, ax = plt.subplots(1, 2)
    # ax[0].imshow(source_image[0, 0, :, :].cpu().detach().numpy())
    # ax[0].plot(matches1[0, :], matches1[1, :], 'r.')
    # ax[1].imshow(target_image[0, 0, :, :].cpu().detach().numpy())
    # ax[1].plot(matches2[0, :], matches2[1, :], 'g.')
    # ax[1].plot(matches1_2[0, :], matches1_2[1, :], 'r.')
    # plt.show()

    # create a dataframe with the matches
    df = pd.DataFrame({'x1': matches1[0, :], 'y1': matches1[1, :],
                       'x2': matches2[0, :], 'y2': matches2[1, :]})
    save_name = 'Dataset/Dataset-processed/' + f'img_{i+500}.csv'
    # print(save_name)
    keypoints_path.append(save_name)
    df.to_csv(save_name, index=False)

Test: 100%|██████████| 100/100 [00:11<00:00,  8.37it/s]


In [14]:
# concat 2 dataframes
df = pd.concat([train_df, test_df], ignore_index=True)
df

,Unnamed: 0,source,Source ROI,target,Target ROI,training,Warped target images,Warped target ROI,Execution time,Directory
0,1337,Dataset/Dataset-processed/10-08-2562/3092607-C...,NaN,Dataset/Dataset-processed/10-08-2562/3092607-C...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/10-08-2562/3092607-C...
1,2957,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...
2,859,Dataset/Dataset-processed/23-12-2559/1426104/b...,NaN,Dataset/Dataset-processed/23-12-2559/1426104/b...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/23-12-2559/1426104/b1
3,3566,Dataset/Dataset-processed/13-01-2560/2446177/c...,NaN,Dataset/Dataset-processed/13-01-2560/2446177/c...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/13-01-2560/2446177/c
4,582,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...
...,...,...,...,...,...,...,...,...,...,...
595,1071,Dataset/Dataset-processed/10-08-2562/3913050-C...,NaN,Dataset/Dataset-processed/10-08-2562/3913050-C...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/10-08-2562/3913050-C...
596,2378,Dataset/Dataset-processed/10-03-2560/2725606-1...,NaN,Dataset/Dataset-processed/10-03-2560/2725606-1...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/10-03-2560/2725606-1...
597,560,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...
598,1926,Dataset/Dataset-processed/10-08-2562/0542330-C...,NaN,Dataset/Dataset-processed/10-08-2562/0542330-C...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/10-08-2562/0542330-C...


In [16]:
# create new column from keypoints_path
df['keypoints_path'] = keypoints_path

In [17]:
df

,Unnamed: 0,source,Source ROI,target,Target ROI,training,Warped target images,Warped target ROI,Execution time,Directory,keypoints_path
0,1337,Dataset/Dataset-processed/10-08-2562/3092607-C...,NaN,Dataset/Dataset-processed/10-08-2562/3092607-C...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/10-08-2562/3092607-C...,Dataset/Dataset-processed/img_0.csv
1,2957,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...,Dataset/Dataset-processed/img_1.csv
2,859,Dataset/Dataset-processed/23-12-2559/1426104/b...,NaN,Dataset/Dataset-processed/23-12-2559/1426104/b...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/23-12-2559/1426104/b1,Dataset/Dataset-processed/img_2.csv
3,3566,Dataset/Dataset-processed/13-01-2560/2446177/c...,NaN,Dataset/Dataset-processed/13-01-2560/2446177/c...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/13-01-2560/2446177/c,Dataset/Dataset-processed/img_3.csv
4,582,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...,Dataset/Dataset-processed/img_4.csv
...,...,...,...,...,...,...,...,...,...,...,...
595,1071,Dataset/Dataset-processed/10-08-2562/3913050-C...,NaN,Dataset/Dataset-processed/10-08-2562/3913050-C...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/10-08-2562/3913050-C...,Dataset/Dataset-processed/img_595.csv
596,2378,Dataset/Dataset-processed/10-03-2560/2725606-1...,NaN,Dataset/Dataset-processed/10-03-2560/2725606-1...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/10-03-2560/2725606-1...,Dataset/Dataset-processed/img_596.csv
597,560,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...,Dataset/Dataset-processed/img_597.csv
598,1926,Dataset/Dataset-processed/10-08-2562/0542330-C...,NaN,Dataset/Dataset-processed/10-08-2562/0542330-C...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/10-08-2562/0542330-C...,Dataset/Dataset-processed/img_598.csv


In [18]:
# remove Unnamed: 0 column
df = df.drop(['Unnamed: 0'], axis=1)
df

,source,Source ROI,target,Target ROI,training,Warped target images,Warped target ROI,Execution time,Directory,keypoints_path
0,Dataset/Dataset-processed/10-08-2562/3092607-C...,NaN,Dataset/Dataset-processed/10-08-2562/3092607-C...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/10-08-2562/3092607-C...,Dataset/Dataset-processed/img_0.csv
1,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/17-03-2560/3788391-1...,Dataset/Dataset-processed/img_1.csv
2,Dataset/Dataset-processed/23-12-2559/1426104/b...,NaN,Dataset/Dataset-processed/23-12-2559/1426104/b...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/23-12-2559/1426104/b1,Dataset/Dataset-processed/img_2.csv
3,Dataset/Dataset-processed/13-01-2560/2446177/c...,NaN,Dataset/Dataset-processed/13-01-2560/2446177/c...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/13-01-2560/2446177/c,Dataset/Dataset-processed/img_3.csv
4,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...,Dataset/Dataset-processed/img_4.csv
...,...,...,...,...,...,...,...,...,...,...
595,Dataset/Dataset-processed/10-08-2562/3913050-C...,NaN,Dataset/Dataset-processed/10-08-2562/3913050-C...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/10-08-2562/3913050-C...,Dataset/Dataset-processed/img_595.csv
596,Dataset/Dataset-processed/10-03-2560/2725606-1...,NaN,Dataset/Dataset-processed/10-03-2560/2725606-1...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/10-03-2560/2725606-1...,Dataset/Dataset-processed/img_596.csv
597,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/03-03-2560/3786536-1...,Dataset/Dataset-processed/img_597.csv
598,Dataset/Dataset-processed/10-08-2562/0542330-C...,NaN,Dataset/Dataset-processed/10-08-2562/0542330-C...,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/10-08-2562/0542330-C...,Dataset/Dataset-processed/img_598.csv


In [6]:
# save the dataframe to a csv file
df.to_csv('Dataset/dataset_eye_actual_keypoints.csv', index=False)

# verify that the saved keypoints are correct